In [ ]:
# Get Both df of annotatore and  make Pivot df
df_ousmane = pd.read_csv(annotations_ousmane_200_csv_path)
df_marco = pd.read_csv(annotations_marco_1000_csv_path)
df_set_pivot = df_ousmane[["author", "sent_id", "text"]].drop_duplicates(subset=["author", "sent_id", "text"])

# Create a matched Dictionnary container
matched_container = {
    'Author': [],
    'SentenceId': [],
    'EVENT': [],
    'STATE': [],
    'ASP-EVENT': [],
    'REP-EVENT': [],
    'MatchedInfo': [],
    'Text': []
}

# Iterate Pivot df and Processing soft matching on Some columns 
for index, row in df_set_pivot.iterrows():
    author = row["author"]
    sent_id = row["sent_id"]
    text = row["text"]
    df_x = df_ousmane.loc[(df_ousmane['author'] == author) & (df_ousmane['sent_id'] == sent_id) & (df_ousmane['text'] == text)]
    df_y = df_marco.loc[(df_marco['author'] == author) & (df_marco['sent_id'] == sent_id) & (df_marco['text'] == text)]
    if (df_x.shape[0] < df_y.shape[0]) :
        df_x, df_y = swapedItems(df_x, df_y)
    for index_x, row_x in df_x.iterrows() :
        items_1 = [row_x["EVENT"], row_x["STATE"], row_x["ASP-EVENT"], row_x["REP-EVENT"]]
        for index_y, row_y in df_y.iterrows() :
            items_2 = [row_y["EVENT"], row_y["STATE"], row_y["ASP-EVENT"], row_y["REP-EVENT"]]
            if items_1 == items_2 : 
                matched_container['Author'].append(author)
                matched_container['SentenceId'].append(sent_id)
                matched_container['Text'].append(text)
                matched_container['EVENT'].append(row_x["EVENT"])
                matched_container['STATE'].append(row_x["STATE"])
                matched_container['ASP-EVENT'].append(row_x["ASP-EVENT"])
                matched_container['REP-EVENT'].append(row_x["REP-EVENT"])
                matched_container['MatchedInfo'].append("Exact Match")
                break
            elif checkContainsRelations(items_1, items_2) :
                matched_container['Author'].append(author)
                matched_container['SentenceId'].append(sent_id)
                matched_container['Text'].append(text)
                if str(row_x["EVENT"]) != "nan" or str(row_y["EVENT"]) != "nan" : 
                    matched_container['EVENT'].append(concateItems(row_x["EVENT"],row_y["EVENT"]))
                else : matched_container['EVENT'].append("")
                if str(row_x["STATE"]) != "nan" or str(row_y["STATE"]) != "nan" : 
                    matched_container['STATE'].append(concateItems(row_x["STATE"], row_y["STATE"]))
                else : matched_container['STATE'].append("")
                if str(row_x["ASP-EVENT"]) != "nan" or str(row_y["ASP-EVENT"]) != "nan" : 
                    matched_container['ASP-EVENT'].append(concateItems(row_x["ASP-EVENT"], row_y["ASP-EVENT"]))
                else : matched_container['ASP-EVENT'].append("")
                if str(row_x["REP-EVENT"]) != "nan" or str(row_y["REP-EVENT"]) != "nan" : 
                    matched_container['REP-EVENT'].append(concateItems(row_x["REP-EVENT"], row_y["REP-EVENT"]))
                else : matched_container['REP-EVENT'].append("")
                matched_container['MatchedInfo'].append("Contains Relation Match")
                break

# Convert the dictionary into DataFrame
df_contains_matched = pd.DataFrame(matched_container, columns = ['SentenceId', 'EVENT', 'STATE', 'ASP-EVENT', 'REP-EVENT', 'MatchedInfo', 'Text'])
df_contains_matched.replace(np.nan, '', regex=True)
df_contains_matched.to_csv(annotations_contains_matched_200_csv_path, index=False)